In [12]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

llm = ChatOpenAI()

In [13]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can be used to create automated tests for software applications, allowing testers to quickly and efficiently run tests and identify any issues or bugs.\n\n2. Test Case Generation: Langsmith can generate test cases based on specific requirements or user stories, helping testers to ensure that all aspects of the software are thoroughly tested.\n\n3. Performance Testing: Langsmith can be used to simulate large numbers of users or data, helping testers to evaluate the performance of the software under different conditions.\n\n4. Regression Testing: Langsmith can automate the process of running regression tests, ensuring that new code changes do not negatively impact existing functionality.\n\n5. Integration Testing: Langsmith can be used to automate the testing of integrated systems or components, helping testers to identify any issues that arise when different parts of the software inter

In [14]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [15]:
chain = prompt | llm 

In [16]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in a variety of ways, thanks to its powerful language analysis capabilities. Here are some ways in which Langsmith can assist with testing:\n\n1. **Automated Testing**: Langsmith can be integrated with testing frameworks to automate the testing of language-specific features in software applications. This can help in detecting issues related to language processing, syntax errors, and other language-related bugs.\n\n2. **Language Compatibility Testing**: Langsmith can analyze code written in different programming languages to ensure compatibility and consistency across different parts of the software. This can help in identifying potential issues that may arise due to language differences.\n\n3. **Code Quality Analysis**: Langsmith can perform static code analysis to identify potential code quality issues, such as code duplication, complexity, and maintainability. This can help in improving the overall quality of the codebase and reducin

In [17]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [18]:
chain = prompt | llm | output_parser

In [19]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can help with testing in several ways:\n\n1. Automated Testing: Langsmith can generate test cases automatically based on the specifications provided, helping to save time and effort in creating test cases manually.\n\n2. Test Coverage: Langsmith can analyze the codebase and identify areas that are not covered by existing test cases, helping to improve overall test coverage.\n\n3. Regression Testing: Langsmith can automatically generate regression test cases to ensure that new code changes do not introduce any unintended bugs or regressions.\n\n4. Performance Testing: Langsmith can help in generating load testing scenarios to simulate high traffic conditions and analyze the performance of the application under stress.\n\n5. Security Testing: Langsmith can assist in generating security test cases to identify vulnerabilities in the application and ensure that sensitive data is protected.\n\nOverall, Langsmith can be a valuable tool in enhancing the testing process by automating

In [20]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [21]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [22]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [24]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

In [25]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [28]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing by allowing users to debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework. It offers tracing capabilities for tracing quick start and evaluation capabilities for evaluation quick start. Users can also learn about the workflows LangSmith supports at each stage of the LLM application lifecycle through the User Guide section.
